This code was adapted from the following github: https://github.com/justjoshtings/satellite_image_segmentation

In [ ]:
!pip install pystac pystac_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 2.7 MB/s eta 0:00:00


##TO -DO:

Change the abspath_curr to reflect Emma's drive/directories, which hase the data

In [ ]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/Application Development Final proj'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import os
import shutil
import math

from PIL import Image
import glob
import cv2
import pickle
import copy

import warnings
import random


import requests

import tempfile
from pathlib import Path
import itertools as it
from urllib.parse import urljoin

from pystac import Item
from pystac.extensions.eo import EOExtension
from pystac.extensions.label import LabelRelType
from pystac.extensions.scientific import ScientificExtension
from pystac_client import Client

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# Ignore warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set matplotlib sizes
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=20)

In [ ]:
# Random Seed

# The random seed
random_seed = 42

# Set random seed in tensorflow
tf.random.set_seed(random_seed)

# Set random seed in numpy
np.random.seed(random_seed)

In [ ]:
# Check what version of TF we are using
print(tf.version.VERSION)

2.12.0


In [ ]:
# Print the number of GPUs available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Test to see if GPU is found and connected
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('\nFound GPU at: {}'.format(device_name))
  print('\nCurrently using:')
  !nvidia-smi -L

Num GPUs Available:  0
GPU device not found


## TO-DO:

Organize data based on Emma's directoires

In [ ]:
meta_df = pd.read_csv(abspath_curr + "/data/metadata.csv")
class_df = pd.read_csv(abspath_curr + "/data/class_dict.csv")

print(class_df.head(10))

print(meta_df.head())

In [ ]:
# 'Valid' does not have masks so let's merge that with the train and take a subset of train for validation
meta_df[meta_df['split'] == 'valid'].head()

In [ ]:
# Update meta_df
meta_df['sat_image_path'] = meta_df['sat_image_path'].str.replace('valid/', 'test/')
meta_df['split'] = meta_df['split'].str.replace('valid', 'test')

In [ ]:
# Create divides for train, test, val
train_df = meta_df[meta_df['split'] == 'train']

train_num_samples = round(len(train_df)*0.9)
val_num_samples = len(train_df) - train_num_samples

In [ ]:
#shuffle all rows of DataFrame
train_df = train_df.sample(frac=1, random_state=random_seed).reset_index(drop=True)
train_df.head()